# Market Research Hack

In [1]:
import itertools
import logging
import random
import sys
import time
from urllib.parse import quote_plus, urlparse

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

logging.basicConfig(
    level=logging.DEBUG,
    format="{asctime} {levelname}: {funcName} - {message}",
    style="{",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)


## Load Original Datasets

In [2]:
df_companies = pd.read_excel("./data/Companies list.xlsx", header=None)
df_DS_keywords = pd.read_excel(
    "./data/Hackathon_Market research_keywords.xlsx", sheet_name="DS"
)
df_DS_keywords.replace({np.nan: ""}, inplace=True)
df_DP_keywords = pd.read_excel(
    "./data/Hackathon_Market research_keywords.xlsx", sheet_name="DP"
)
df_DP_keywords.replace({np.nan: ""}, inplace=True)
df_AD_keywords = pd.read_excel(
    "./data/Hackathon_Market research_keywords.xlsx", sheet_name="AD"
)
df_AD_keywords.replace({np.nan: ""}, inplace=True)


## Prepare Utilities and Variables

In [3]:
def bing_search(query: str, user_agent: str, page: int = 0):
    BASE_URL = "https://www.bing.com/search?q="
    headers = {"user-agent": user_agent}
    request = requests.get(
        f"{BASE_URL}{quote_plus(query)}&first={(page * 10) + 1}", headers=headers
    )

    results = list()
    if request.status_code == 200:
        soup = BeautifulSoup(request.content, "html.parser")
        try:
            for x in soup.find_all("li", {"class": "b_algo"}):
                results.append(x.find_all("a")[0]["href"])
        except Exception as ex:
            logging.error(f"Exception: {ex}")
    else:
        logging.error(f"HTTP Status: {request.status_code}")

    return results


n_pages = 5
user_agent = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0"
)

companies = df_companies[1].to_list()

df_DS_keywords["group"] = "drug substance"
df_DP_keywords["group"] = "drug package"
df_AD_keywords["group"] = "analytical development"
keywords = list()
for _, x in itertools.chain(
    df_DS_keywords.iterrows(), df_DP_keywords.iterrows(), df_AD_keywords.iterrows()
):
    tmp = x.to_dict()
    keywords.append(
        dict(
            keyword=tmp["keyword"],
            synonyms=[y.strip() for y in tmp["synonyms"].split(",")],
            hyponyms=[y.strip() for y in tmp["hyponyms"].split(",")],
            group=tmp["group"],
        )
    )

blacklist = list()
# blacklisted domains MUST NOT have http:// or https:// before the FQDN
# e.g. wikipedia.com, www.wikipedia.com, www.ncbi.nlm.nih.gov
try:
    with open("domain_blacklist.txt", "r") as f:
        blacklist = f.read().splitlines()
except FileNotFoundError:
    logging.warning("domain_blacklist.txt is missing")

if not blacklist:
    logging.info("domain_blacklist.txt is empty")


## Company homepage domain search

### Proof Of Concept

In [4]:
scraped_websites = set()
for i in range(n_pages):
    # go through the first X pages of results
    # sleep random time to avoid being detected
    time.sleep(random.randrange(5, 15))
    scraped_websites.update(
        [
            url
            for url in bing_search(
                query="API prefer:GMP prefer:company prefer:pharma prefer:drug prefer:packaging language:en",
                user_agent=user_agent,
                page=i,
            )
        ]
    )

# exclude blacklisted domains
filtered_websites = [
    dict(domain=urlparse(url=url).netloc, url=url)
    for url in scraped_websites
    if urlparse(url=url).netloc not in blacklist
]
df = pd.DataFrame(
    dict(
        domain=list(map(lambda x: x["domain"], filtered_websites)),
        url=list(map(lambda x: x["url"], filtered_websites)),
    )
)
df.sort_values(by=["domain", "url"], inplace=True)
df.to_csv("domains.csv", index=False)
df


2022-04-30 18:29:18 DEBUG: _new_conn - Starting new HTTPS connection (1): www.bing.com:443
2022-04-30 18:29:18 DEBUG: _make_request - https://www.bing.com:443 "GET /search?q=syringe+prefer%3Acompany+prefer%3Apharma+prefer%3Adrug+prefer%3Apackaging+language%3Aen&first=1 HTTP/1.1" 200 None
2022-04-30 18:29:24 DEBUG: _new_conn - Starting new HTTPS connection (1): www.bing.com:443
2022-04-30 18:29:24 DEBUG: _make_request - https://www.bing.com:443 "GET /search?q=syringe+prefer%3Acompany+prefer%3Apharma+prefer%3Adrug+prefer%3Apackaging+language%3Aen&first=11 HTTP/1.1" 200 None
2022-04-30 18:29:32 DEBUG: _new_conn - Starting new HTTPS connection (1): www.bing.com:443
2022-04-30 18:29:32 DEBUG: _make_request - https://www.bing.com:443 "GET /search?q=syringe+prefer%3Acompany+prefer%3Apharma+prefer%3Adrug+prefer%3Apackaging+language%3Aen&first=21 HTTP/1.1" 200 None
2022-04-30 18:29:46 DEBUG: _new_conn - Starting new HTTPS connection (1): www.bing.com:443
2022-04-30 18:29:46 DEBUG: _make_request

,domain,url
18,airekacells.com,https://airekacells.com/blog/syringe-filter
3,avvocatotributariomilano.mi.it,https://avvocatotributariomilano.mi.it/Syringe...
19,boshibishi.comuni.fvg.it,https://boshibishi.comuni.fvg.it/10_Ml_Syringe...
23,comuni.fvg.it,https://comuni.fvg.it/Remedi_Relax_Indica_Syri...
1,dictionary.cambridge.org,https://dictionary.cambridge.org/dictionary/en...
20,escapefromtarkov.fandom.com,https://escapefromtarkov.fandom.com/wiki/Dispo...
30,ftb.fandom.com,https://ftb.fandom.com/wiki/Syringe_(RFTools)
8,gpcmedicalequipment.wordpress.com,https://gpcmedicalequipment.wordpress.com/2015...
12,grabcad.com,https://grabcad.com/library/tag/syringe#!
5,gtfo.fandom.com,https://gtfo.fandom.com/wiki/IIX_Syringe


### Actual Code

In [5]:
scraped_websites = dict(
    drug_substance=set(), drug_package=set(), analytical_development=set()
)

for x in keywords:
    query = (
        x["keyword"]
        + " prefer:company prefer:pharma "
        + " ".join([f"prefer:{group_word}" for group_word in x["group"].split()])
        + " language:en"
    )
    # go through the first X pages of results
    for i in range(n_pages):
        # sleep random time to avoid being detected
        time.sleep(random.randrange(5, 15))
        scraped_websites[x["group"].replace(" ", "_")].update(
            [
                url
                for url in bing_search(
                    query=query,
                    user_agent=user_agent,
                    page=i,
                )
            ]
        )

for group, websites in scraped_websites.items():
    # exclude blacklisted domains
    filtered_websites = [
        dict(domain=urlparse(url=url).netloc, url=url)
        for url in websites
        if urlparse(url=url).netloc not in blacklist
    ]
    df = pd.DataFrame(
        dict(
            domain=list(map(lambda x: x["domain"], filtered_websites)),
            url=list(map(lambda x: x["url"], filtered_websites)),
        )
    )
    df.sort_values(by=["domain", "url"], inplace=True)
    df.to_csv(f"domains_{group}.csv", index=False)


2022-05-03 16:04:18 DEBUG: _new_conn - Starting new HTTPS connection (1): www.bing.com:443
2022-05-03 16:04:18 DEBUG: _make_request - https://www.bing.com:443 "GET /search?q=molecule+stability+prefer%3Acompany+prefer%3Apharma+prefer%3Adrug+prefer%3Asubstance+language%3Aen&first=1 HTTP/1.1" 200 None
2022-05-03 16:04:25 DEBUG: _new_conn - Starting new HTTPS connection (1): www.bing.com:443
2022-05-03 16:04:25 DEBUG: _make_request - https://www.bing.com:443 "GET /search?q=molecule+stability+prefer%3Acompany+prefer%3Apharma+prefer%3Adrug+prefer%3Asubstance+language%3Aen&first=11 HTTP/1.1" 200 None
2022-05-03 16:04:36 DEBUG: _new_conn - Starting new HTTPS connection (1): www.bing.com:443
2022-05-03 16:04:36 DEBUG: _make_request - https://www.bing.com:443 "GET /search?q=molecule+stability+prefer%3Acompany+prefer%3Apharma+prefer%3Adrug+prefer%3Asubstance+language%3Aen&first=21 HTTP/1.1" 200 None
2022-05-03 16:04:45 DEBUG: _new_conn - Starting new HTTPS connection (1): www.bing.com:443
2022-05

## Keyword extraction 

In [ ]:
domains = pd.read_csv("domains.csv")

# TODO - PERFORM ACTUAL KEYWORD EXTRACTION
df = pd.DataFrame(
    {
        "keywords": [["molecules", "gene"], ["psyshiary", "health"]],
        "company_name": ["lonza", "Boehringer Ingelheim"],
    },
    index=domains["domain"],
)
df.index.name = "domain"

df.to_csv("extracted_keywords.csv")
df


,keywords,company_name
domain,,
https://www.lonza.com/,"[molecules, gene]",lonza
https://www.boehringer-ingelheim.com/,"[psyshiary, health]",Boehringer Ingelheim


## Area of works classification

In [3]:
keywords = pd.read_csv("extracted_keywords.csv")


def classifier_predict(keywords):
    # TODO - return actual trained-classifier predictions
    return {"ad": 0.2, "ds": 0.2, "dp": 0.6}


predictions = keywords["keywords"].map(lambda keywords: classifier_predict(keywords))

df = pd.DataFrame({"predictions": predictions.to_list()}, index=domains["domain"])
df.index.name = "domain"

df.to_csv("area_of_works_predictions.csv")

df


,predictions
domain,
https://www.lonza.com/,"{'ad': 0.2, 'ds': 0.2, 'dp': 0.6}"
https://www.boehringer-ingelheim.com/,"{'ad': 0.2, 'ds': 0.2, 'dp': 0.6}"


## Interactive user interface 

In [4]:
keywords = pd.read_csv("extracted_keywords.csv")
area_of_works_predictions = pd.read_csv("area_of_works_predictions.csv")


"""
TODO
show a UI with buttons with functionality for flagging domains (which writes to domains_blacklist.txt) and
flagging kywords (which writes to keywords_blacklist.txt)
"""
pd.merge(keywords, area_of_works_predictions)


,domain,keywords,company_name,predictions
0,https://www.lonza.com/,"['molecules', 'gene']",lonza,"{'ad': 0.2, 'ds': 0.2, 'dp': 0.6}"
1,https://www.boehringer-ingelheim.com/,"['psyshiary', 'health']",Boehringer Ingelheim,"{'ad': 0.2, 'ds': 0.2, 'dp': 0.6}"
